# Speech to Text with Open AI Whisper

In [1]:
# %pip install -U openai-whisper

In [2]:
import gradio as gr
import numpy as np
import pytube
import time
import sys
import whisper

In [3]:
sys.version

'3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]'

## Download a YouTube audio mp4 file

In [4]:
video = "https://www.youtube.com/watch?v=mRTqaC04T04"

yt = pytube.YouTube(video)
audio = yt.streams.get_audio_only()
audio.download()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/standardd14sr/code/Users/seretkow/Speech to text/Sur les routes de linnovation - Economie Circulaire - Déchetterie commune POINTP.mp4'

In [5]:
os.rename(
    "Sur les routes de linnovation - Economie Circulaire - Déchetterie commune POINTP.mp4",
    "audio.mp4",
)

## Base model

In [6]:
model = whisper.load_model("base")

In [7]:
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is multilingual and has 71,825,920 parameters.


In [8]:
model

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-5): 6 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=512, out_features=512, bias=True)
          (key): Linear(in_features=512, out_features=512, bias=False)
          (value): Linear(in_features=512, out_features=512, bias=True)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (attn_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (mlp_ln): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((512,), eps=1e-05,

In [9]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("audio.mp4")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

In [10]:
# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")
print(f"Confidence = {max(probs.values())}")

Detected language: fr
Confidence = 0.9830532670021057


## Extract mp3 audio file from the audio mp4

In [11]:
from moviepy.editor import AudioFileClip

# Load the audio file
audio = AudioFileClip("audio.mp4")
sample = audio.subclip(20, 100)

# Save the new audio file
sample.write_audiofile("audio.mp3")

ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: N

MoviePy - Writing audio in audio.mp3


MoviePy - Done.


In [12]:
sample.write_audiofile("audio.wav")

MoviePy - Writing audio in audio.wav


MoviePy - Done.


In [13]:
from IPython.display import Audio

Audio("audio.mp3", autoplay=False)

In [14]:
start = time.time()
text_base = model.transcribe("audio.mp3")
elapsed = time.time() - start
print(
    "Elapsed time: "
    + time.strftime(
        "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
    )
)

Elapsed time: 00:00:07.922408


In [15]:
from pprint import pprint

pprint(text_base, indent=2, width=30)

{ 'language': 'fr',
  'segments': [ { 'avg_logprob': -0.2834169968314793,
                  'compression_ratio': 1.7526132404181185,
                  'end': 1.0,
                  'id': 0,
                  'no_speech_prob': 0.12293168157339096,
                  'seek': 0,
                  'start': 0.0,
                  'temperature': 0.0,
                  'text': ' '
                          'Bonjour '
                          'Patrick '
                          '!',
                  'tokens': [ 50364,
                              25431,
                              13980,
                              2298,
                              50414]},
                { 'avg_logprob': -0.2834169968314793,
                  'compression_ratio': 1.7526132404181185,
                  'end': 2.0,
                  'id': 1,
                  'no_speech_prob': 0.12293168157339096,
                  'seek': 0,
                  'start': 1.0,
                  'temperature': 0.0,
       

In [16]:
text_base["text"]

" Bonjour Patrick ! Bonjour ! Alors vous êtes le responsable de ce centre de collecte, comment va l'activité ? La activité va très bien, c'est vraiment satisfait et le val et client avec le sourire. C'est content de la prestation que vous offrez, ils sont satisfaits aussi des plages d'ouverture et des types de filières que vous leur proposer. Les clients sont hyper satisfaits, je sais aussi que mes agents sont vraiment satisfaits de venir travailler ici tous les matins. Bonjour à tous les deux. Dis-t-nous en quoi le service au fer par cette déchétrie vous aide aux clients dans votre métier ? On est spécialisé dans la innovation, ce qui nous intéresse, c'est de pouvoir tout mélanger et puis venir ici faire le tri et un gain de temps énorme, de pouvoir recharger nos camions, racheter. Et oui parce que cette déchétrie est partagée entre point P et la plateforme ? Tout à fait, oui, voilà. C'est très convétitif, puisque comme il y a des parties qui sont gratuite. On utilise beaucoup parce q

## Large model

In [17]:
# Loading large model
model = whisper.load_model("large")

In [18]:
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is multilingual and has 1,541,384,960 parameters.


In [19]:
model

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1280, out_features=1280, bias=True)
          (key): Linear(in_features=1280, out_features=1280, bias=False)
          (value): Linear(in_features=1280, out_features=1280, bias=True)
          (out): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1280, out_features=5120, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (mlp_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((

In [20]:
start = time.time()
text_large = model.transcribe("audio.mp3")
elapsed = time.time() - start
print(
    "Elapsed time: "
    + time.strftime(
        "%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:15], time.gmtime(elapsed)
    )
)

Elapsed time: 00:01:18.523413


In [21]:
pprint(text_large, indent=2, width=30)

{ 'language': 'fr',
  'segments': [ { 'avg_logprob': -0.23453645408153534,
                  'compression_ratio': 1.7288732394366197,
                  'end': 1.0,
                  'id': 0,
                  'no_speech_prob': 0.022877629846334457,
                  'seek': 0,
                  'start': 0.0,
                  'temperature': 0.0,
                  'text': ' '
                          'Bonjour '
                          'Patrick '
                          '!',
                  'tokens': [ 50364,
                              25431,
                              13980,
                              2298,
                              50414]},
                { 'avg_logprob': -0.23453645408153534,
                  'compression_ratio': 1.7288732394366197,
                  'end': 2.0,
                  'id': 1,
                  'no_speech_prob': 0.022877629846334457,
                  'seek': 0,
                  'start': 1.0,
                  'temperature': 0.0,
   

In [22]:
pprint(text_large["text"])

(' Bonjour Patrick ! Bonjour ! Alors vous êtes le responsable de ce centre de '
 "collecte, comment va l'activité ? L'activité va très bien, je suis vraiment "
 'satisfait de voir les clients avec le sourire. Vous êtes content de la '
 'prestation que vous offrez ? Ils sont satisfaits eux aussi des plages '
 "d'ouverture et des types de filières que vous leur proposez ? Les clients "
 'sont hyper satisfaits, je sais aussi que mes agences sont vraiment '
 'satisfaites de venir travailler ici tous les matins. Bonjour à tous les deux '
 '! Dites-nous en quoi le service offert par cette déchetterie vous aide au '
 'quotidien dans votre métier ? On est spécialisé dans la rénovation, donc ce '
 "qui nous intéresse c'est de pouvoir tout mélanger et venir ici faire le tri. "
 'Et un gain de temps énorme de pouvoir recharger nos camions, racheter. Et '
 'oui parce que cette déchetterie est partagée entre Point P et la plateforme '
 "? Tout à fait oui, c'est très compétitif puisque comme il y a 

In [23]:
pprint(text_base["text"])

(' Bonjour Patrick ! Bonjour ! Alors vous êtes le responsable de ce centre de '
 "collecte, comment va l'activité ? La activité va très bien, c'est vraiment "
 "satisfait et le val et client avec le sourire. C'est content de la "
 "prestation que vous offrez, ils sont satisfaits aussi des plages d'ouverture "
 'et des types de filières que vous leur proposer. Les clients sont hyper '
 'satisfaits, je sais aussi que mes agents sont vraiment satisfaits de venir '
 'travailler ici tous les matins. Bonjour à tous les deux. Dis-t-nous en quoi '
 'le service au fer par cette déchétrie vous aide aux clients dans votre '
 "métier ? On est spécialisé dans la innovation, ce qui nous intéresse, c'est "
 'de pouvoir tout mélanger et puis venir ici faire le tri et un gain de temps '
 'énorme, de pouvoir recharger nos camions, racheter. Et oui parce que cette '
 'déchétrie est partagée entre point P et la plateforme ? Tout à fait, oui, '
 "voilà. C'est très convétitif, puisque comme il y a des parti

In [24]:
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is multilingual and has 1,541,384,960 parameters.


## Gradio webapp

In [25]:
def transcribe_audio(audio_file):
    """
    Gradio function
    """
    model = whisper.load_model("base")
    result = model.transcribe(audio_file)

    return result["text"]

In [26]:
audio_input = gr.inputs.Audio(source="upload", type="filepath")
output_text = gr.outputs.Textbox()

logo = "https://github.com/retkowsky/images/blob/master/logoazureCV.png?raw=true"
image = "<center> <img src= {} width=50px></center>".format(logo)

webapp = gr.Interface(
    fn=transcribe_audio,
    inputs=audio_input,
    outputs=output_text,
    description=image,
    title="Audio Transcription App with Open AI Whisper",
)

In [27]:
webapp.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://abb6dc81d481f2542c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
